In [2]:
import numpy as np
import os

In [3]:
headtype=">u4,>u2,(2)>u1,>u8,S128"
nt=128
nx=16

In [80]:
# lime格式分为头部和数据，头部由18个int64的数据段组成，具体可以看lime_binary_header.h。第二个int64存放数据段的长度。该长度需要补满整个int64的长度。
def get_new_offset(f,offset):
    header = np.fromfile(f,dtype=">u1",count=8*18,offset=offset)
    header.dtype=headtype
    print(header[0][3])
    print(header[0][4])
    headpad = 8 - np.int(header[0][3]%8) if header[0][3] > 0 else 0
    new_offset = np.int(header[0][3] + headpad)
    return new_offset
def get_message(f,offset=0,datalength=160):
    message = np.fromfile(f,dtype="S%i"%datalength,count=1,offset=8*18)
    print(message)
    return message

In [96]:
#由于头部的数据都可以看成是int64的整数倍，可以方便的算出组态数据的起始位置
def readlimedata(f,nt=128,nx=16):
    data = np.fromfile(f,dtype=">f8")
    num_data = nx**3 * nt * 3 * 3 * 4 * 2 
    return data[-num_data-18-17:-18-17]

In [134]:
# lime格式数据的顺序为(nt,nz,ny,nx,mu,ic1,ic2,ri)
# clqcd常用的格式为(imu,ic2,ic1,ir,nt,nz,ny,nx)
def tran_to_lime(u):
    u = u.reshape(nt,nx,nx,nx,4,3,3,2)
    tmp = np.zeros((nt,nx,nx,nx,4,3,3),"c16")
    tmp = u[...,0]+1j*u[...,1]
    return tmp
def lime_tran_to_clqcd(u):
    u = u.reshape(nt,nx,nx,nx,4,3,3,2)
    u = np.transpose(u,(4,6,5,7,0,1,2,3))
    tmp = np.zeros((4,3,3,nt,nx,nx,nx),"c16")
    tmp = u[:,:,:,0,...]+1j*u[:,:,:,1,...]
    return tmp
def cal_plaq(u,i,j):
    tmp = u
    plaq_1 = np.einsum("ij...,jk...->ik...",tmp[i,:,:,:,:,:,:],np.roll(tmp[j,:,:,:,:,:,:],-1,axis=5-i))
    #plaq_1 = np.tensordot(tmp[i,:,:,:,:,:,:],np.roll(tmp[j,:,:,:,:,:,:],-1,axis=5),axes=([1],[0]))
    tmp_dag = np.conjugate(np.transpose(tmp,(0,2,1,3,4,5,6)))
    plaq_2 = np.einsum("ij...,jk...->ik...",np.roll(tmp_dag[i,:,:,:,:,:,:],-1,axis=5-j),tmp_dag[j,:,:,:,:,:,:])
    #plaq_2 = np.tensordot(np.roll(tmp[i,:,:,:,:,:,:],-1,axis=4),tmp_dag[j,:,:,:,:,:,:],axes=([1],[0]))
    plaq = np.einsum("ij...,ji...->...",plaq_1,plaq_2)
    #plaq = np.tensordot(plaq_1,plaq_2,axes=([1],[0]))
    return plaq
def test_u(u):
    tmp_u = u[0,:,:,0,0,0,0]
    tmp_udag = np.conjugate(np.transpose(tmp_u,(1,0)))
    print("u times u^dag = \n",np.dot(tmp_u,tmp_udag))
    tmp_inv = np.linalg.inv(tmp_u)
    print("u u^dag and u_inv \n",tmp_u,"\n",tmp_udag,"\n",tmp_inv)

In [7]:
path = "/CLQCD/configs/clqcd/charm.b28.16_128.wo_stout.corrected"
configs = ["s1.0_cfg_%d.lime"%i for i in range(1,101)]

In [8]:
files = [path + "/" + i for i in configs]

In [128]:
data_1 = readlimedata(files[0])

In [145]:
data_1_clqcd = lime_tran_to_clqcd(data_1)
plaq = cal_plaq(data_1_clqcd,0,1)
np.average(plaq)

(0.1592197417401985-0.0014368456913381863j)

In [130]:
data_2 = np.fromfile(path+"/tmp.dat",">f8")

In [135]:
data_2_lime = tran_to_lime(data_2)

In [138]:
tmp = data_2_lime[0,0,0,0,0,:,:]
np.dot(tmp,np.conj(np.transpose(tmp,(1,0))))

array([[ 1.00000000e+00+1.86871646e-17j, -5.55111512e-17-5.55111512e-17j,
         1.11022302e-16+2.77555756e-17j],
       [-5.55111512e-17+8.32667268e-17j,  1.00000000e+00-1.48448759e-18j,
        -2.77555756e-17+2.08166817e-17j],
       [ 1.11022302e-16+0.00000000e+00j, -2.77555756e-17-2.08166817e-17j,
         1.00000000e+00+7.38577394e-18j]])

In [139]:
data_2_clqcd = lime_tran_to_clqcd(data_2)

In [140]:
data_2_clqcd.shape

(4, 3, 3, 128, 16, 16, 16)

In [142]:
tmp = data_2_clqcd[0,:,:,0,0,0,0]
np.dot(tmp,np.conj(np.transpose(tmp,(1,0))))

array([[ 1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -2.77555756e-17-2.77555756e-17j],
       [ 0.00000000e+00+0.00000000e+00j,  1.00000000e+00+0.00000000e+00j,
         0.00000000e+00-2.42861287e-17j],
       [-2.77555756e-17+2.77555756e-17j,  0.00000000e+00+2.42861287e-17j,
         1.00000000e+00+0.00000000e+00j]])

In [143]:
data_1_lime.shape

(4, 3, 3, 128, 16, 16, 16)

In [146]:
tmp = data_1_clqcd[0,:,:,0,0,0,0]
np.dot(tmp,np.conj(np.transpose(tmp,(1,0))))

array([[ 1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -2.77555756e-17-2.77555756e-17j],
       [ 0.00000000e+00+0.00000000e+00j,  1.00000000e+00+0.00000000e+00j,
         0.00000000e+00-2.42861287e-17j],
       [-2.77555756e-17+2.77555756e-17j,  0.00000000e+00+2.42861287e-17j,
         1.00000000e+00+0.00000000e+00j]])

In [147]:
test_u(data_1_clqcd)

u times u^dag = 
 [[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -2.77555756e-17-2.77555756e-17j]
 [ 0.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
   0.00000000e+00-2.42861287e-17j]
 [-2.77555756e-17+2.77555756e-17j  0.00000000e+00+2.42861287e-17j
   1.00000000e+00+0.00000000e+00j]]
u u^dag and u_inv 
 [[-0.25866568+0.05972229j -0.31542758+0.90355163j  0.01284417+0.11601823j]
 [ 0.18878178-0.22077907j  0.10531955-0.00561475j -0.31038697+0.89897399j]
 [-0.71636896+0.57615872j  0.00144147-0.27013715j -0.16684554+0.23247162j]] 
 [[-0.25866568-0.05972229j  0.18878178+0.22077907j -0.71636896-0.57615872j]
 [-0.31542758-0.90355163j  0.10531955+0.00561475j  0.00144147+0.27013715j]
 [ 0.01284417-0.11601823j -0.31038697-0.89897399j -0.16684554-0.23247162j]] 
 [[-0.25866568-0.05972229j  0.18878178+0.22077907j -0.71636896-0.57615872j]
 [-0.31542758-0.90355163j  0.10531955+0.00561475j  0.00144147+0.27013715j]
 [ 0.01284417-0.11601823j -0.31038697-0.89897399j -0

In [152]:
np.average(cal_plaq(data_2_clqcd,0,1))

(0.1592197417401985-0.0014368456913381863j)

u times u^dag = 
 [[ 1.61059606+0.j -0.5962468 +0.j  0.40859662+0.j]
 [-0.5962468 +0.j  0.50900961+0.j -0.19897855+0.j]
 [ 0.40859662+0.j -0.19897855+0.j  0.3144044 +0.j]]
u u^dag and u_inv 
 [[ 0.16164105+0.16164105j -0.58539047-0.58539047j -0.66067254-0.66067254j]
 [-0.04919479-0.04919479j  0.50204873+0.50204873j -0.00563522-0.00563522j]
 [ 0.35238082+0.35238082j -0.16450459-0.16450459j -0.07725413-0.07725413j]] 
 [[ 0.16164105-0.16164105j -0.04919479+0.04919479j  0.35238082-0.35238082j]
 [-0.58539047+0.58539047j  0.50204873-0.50204873j -0.16450459+0.16450459j]
 [-0.66067254+0.66067254j -0.00563522+0.00563522j -0.07725413+0.07725413j]] 
 [[-0.18300201+0.18300201j  0.29243483-0.29243483j  1.54369047-1.54369047j]
 [-0.02666411+0.02666411j  1.01528005-1.01528005j  0.15397122-0.15397122j]
 [-0.77795231+0.77795231j -0.8280438 +0.8280438j   0.24125487-0.24125487j]]


In [81]:
for i in range(10):
    offset = 0
    get_new_offset(files[0],offset=i*18 + offset)

154
b'scidac-private-file-xml'


160

In [90]:
np.einsum?

Signature: np.einsum(*operands, **kwargs)
Docstring:
einsum(subscripts, *operands, out=None, dtype=None, order='K',
       casting='safe', optimize=False)

Evaluates the Einstein summation convention on the operands.

Using the Einstein summation convention, many common multi-dimensional,
linear algebraic array operations can be represented in a simple fashion.
In *implicit* mode `einsum` computes these values.

In *explicit* mode, `einsum` provides further flexibility to compute
other array operations that might not be considered classical Einstein
summation operations, by disabling, or forcing summation over specified
subscript labels.

See the notes and examples for clarification.

Parameters
----------
subscripts : str
    Specifies the subscripts for summation as comma separated list of
    subscript labels. An implicit (classical Einstein summation)
    calculation is performed unless the explicit indicator '->' is
    included as well as subscript labels of the precise output fo